In [3]:
import struct
import datetime
from zoneinfo import ZoneInfo

def read_mp4_creation_time(path):
    with open(path, 'rb') as f:
        while True:
            header = f.read(8)
            if len(header) < 8:
                return None  # no moov/mvhd found

            atom_size, atom_type = struct.unpack('>I4s', header)
            atom_type = atom_type.decode('utf-8')

            if atom_type == 'moov':
                moov_end = f.tell() + atom_size - 8

                while f.tell() < moov_end:
                    sub_header = f.read(8)
                    if len(sub_header) < 8:
                        return None

                    sub_size, sub_type = struct.unpack('>I4s', sub_header)
                    sub_type = sub_type.decode('utf-8')

                    if sub_type == 'mvhd':
                        version = f.read(1)
                        f.read(3)  # flags

                        if version == b'\x00':
                            # 32‑bit creation time
                            creation_time = struct.unpack('>I', f.read(4))[0]
                        else:
                            # 64‑bit creation time
                            creation_time = struct.unpack('>Q', f.read(8))[0]

                        # MP4 epoch is 1904‑01‑01
                        mp4_epoch = datetime.datetime(1904, 1, 1, tzinfo=datetime.timezone.utc)
                        return mp4_epoch + datetime.timedelta(seconds=creation_time)

                    # skip to next sub‑atom
                    f.seek(sub_size - 8, 1)

            else:
                # skip this top-level atom
                f.seek(atom_size - 8, 1)


In [ ]:
video_name = r"GX0101061222.MP4"
video_path = rf"C:\Users\232078\OneDrive - Weir Group PLC\Documents\India_test_data\{video_name}"
timestamp = read_mp4_creation_time(video_path)
ist_time = timestamp.astimezone(ZoneInfo("Asia/Kolkata"))

print(f"date listed: {video_name[-8:-4]}")
print(f"Creation date (IST) for {video_name}: {ist_time}")



video_name = r"GX0101071242.MP4"
video_path = rf"C:\Users\232078\OneDrive - Weir Group PLC\Documents\India_test_data\{video_name}"
timestamp = read_mp4_creation_time(video_path)
ist_time = timestamp.astimezone(ZoneInfo("Asia/Kolkata"))

print(f"date listed: {video_name[-8:-4]}")
print(f"Creation date (IST) for {video_name}: {ist_time}")

date listed: 1222
Creation date (IST) for GX0101061222.MP4: 2026-02-11 12:22:36+05:30
date listed: 1639
Creation date (IST) for GX0201041639.MP4: 2026-02-09 15:59:58+05:30


# Discrepancies

Creation date (IST) for GX0201041639.MP4: 2026-02-09 15:59:58+05:30
<br>
1639 -> 15:59:58
<br>
Possible file mismatch: GX0101041559 (1559)
<br>
<br>



